In [1]:
from jax import config
config.update('jax_enable_x64', True)
from pathlib import Path

import jax
import gpjax as gpx
import jax.numpy as jnp
import jax.random as jr
import matplotlib.pyplot as plt

from uncprop.models.vsem.experiment import summarize_rep, load_results
from uncprop.utils.plot import (
    set_plot_theme, 
    smart_subplots,
    plot_coverage_curve_reps,
)

colors = set_plot_theme()
base_dir = Path('/Users/andrewroberts/Desktop/git-repos/bip-surrogates-paper')
output_dir = base_dir / 'out' / 'final'

/Users/andrewroberts/Desktop/git-repos/bip-surrogates-paper/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# read experiment output

out_dir = base_dir / 'out' / 'vsem'
gp_subdirs = ['gp_N4', 'gp_N8', 'gp_N16']
clip_gp_subdirs = ['clip_gp_N4', 'clip_gp_N8', 'clip_gp_N16']

gp_results = load_results(out_dir, gp_subdirs)
clip_gp_results = load_results(out_dir, clip_gp_subdirs)

results = {'gp': gp_results, 'clip_gp': clip_gp_results}

### Coverage Plots

In [ ]:
ndesign = [4, 8, 16]
approx = ['mean', 'eup', 'ep']
surrogate_labels = ['gp', 'clip_gp']
plots = {}

for lbl in surrogate_labels:
    surrogate_results = results[lbl]
    plots[lbl] = smart_subplots(nrows=3, ncols=3, flatten=False)
    fig, axs = plots[lbl]

    for n_idx, n in enumerate(ndesign):
        subdir_name = f'{lbl}_N{n}'
        res = results[lbl][subdir_name]['results']
        for dist_idx, dist_name in enumerate(approx):
            ax = axs[n_idx, dist_idx]
            idx = list(res['dist_names']).index(dist_name)

            plot_coverage_curve_reps(log_coverage=res['log_coverage'][:,[idx],:], 
                                    probs=res['probs'], 
                                    names=[dist_name],
                                    colors=colors,
                                    qmin=0.05,
                                    qmax=0.95,
                                    single_plot=True,
                                    ax=ax,
                                    alpha=0.2, max_cols=3)
            
            if n_idx == 0: # first row
                ax.set_title(dist_name, fontsize=ax.title.get_fontsize() * 2)
            if n_idx != (len(ndesign)-1): # all but final row
                ax.set_xlabel(None)
                # ax.set_xticklabels([])
            else:
                ax.set_xlabel('nominal', fontsize=ax.xaxis.label.get_size() * 2)
            if dist_idx != 0: # all but first column
                ax.set_ylabel(None)
                # ax.set_yticklabels([])
            else:
                ax.set_ylabel(f'actual (N = {n})', fontsize=ax.yaxis.label.get_size() * 2)

            ax.get_legend().remove()


plots['gp'][0].savefig(output_dir / 'vsem_coverage_gp.png',
                       bbox_inches='tight')
plots['clip_gp'][0].savefig(output_dir / 'vsem_coverage_clip_gp.png',
                            bbox_inches='tight')

display(plots['gp'][0])
display(plots['clip_gp'][0])


In [ ]:
from uncprop.utils.grid import plot_coverage_curve_reps

for subdir, info in gp_results.items():
    res = info['results']
    
    # top row
    set_title = (subdir == 'clip_gp_')

    if subdir == 'clip_gp_N4':
        set_title = True

    plot_coverage = plot_coverage_curve_reps(log_coverage=res['log_coverage'], 
                                             probs=res['probs'], 
                                             names=res['dist_names'],
                                             colors=colors,
                                             qmin=0.05,
                                             qmax=0.95,
                                             single_plot=False,
                                             alpha=0.3, max_cols=3)

## Investigate specific reps

In [ ]:
subdir_name = 'clip_gp_N4'
rep_idx = 4

grid, results, log_dens, plots = summarize_rep(out_dir=out_dir,
                                               subdir_name=subdir_name,
                                               rep_idx=rep_idx,
                                               n_reps=100)

# TEMP

In [ ]:
from uncprop.core.surrogate import Surrogate, SurrogateDistribution
from uncprop.core.samplers import init_cut_kernel, mcmc_loop
from uncprop.core.distribution import Distribution, GaussianFromNumpyro
from numpyro.distributions import MultivariateNormal

key = jr.key(2342343)

In [ ]:
class TestDist(Distribution):

    def __init__(self, input):
        self.input = input

    @property
    def dim(self) -> int:
        return 1

    @property
    def support(self) -> tuple[tuple, tuple] | None:
        """
        Should return tuple of the form lower, upper where lower and upper
        are each tuples of length `dim` giving the dimension-by-dimension
        lower and upper bounds, respectively. Should be `-jnp.inf` or 
        `jnp.inf` for unbounded dimensions. None is interpreted as unconstrained.
        """
        return None
    
    def sample(self, key, n: int = 1):
        return jnp.array(-0.5 * self.input ** 2)


class TestSurrogate(Surrogate):
    def __call__(self, input):
        return TestDist(input)
    
    @property
    def input_dim(self) -> int:
        return 1


class TestSurrogateDist(SurrogateDistribution):

    def __init__(self, surrogate):
        self._surrogate = surrogate

    @property
    def surrogate(self) -> Surrogate:
        return self._surrogate

    @property
    def dim(self) -> int:
        return 1

    @property
    def support(self) -> tuple[tuple, tuple] | None:
        return None
    
    def log_density(self, input):
        return self.surrogate(input)

In [ ]:
surrogate = TestSurrogate()
dist = TestSurrogateDist(surrogate)

key_init, key_sample = jr.split(key, 2)

initial_state, kernel = init_cut_kernel(key=key_init,
                                        surrogate_dist=dist,
                                        initial_position=jnp.array([0.0]),
                                        u_prop_cov=jnp.array([[1.0]]))

In [ ]:
out = mcmc_loop(key=key_sample,
                kernel=kernel,
                initial_state=initial_state,
                num_samples=100_000)

### Test with VSEM

In [ ]:
def _summarize_rep_gp(rep):
    exact = rep.density_comparison.log_dens_grid['exact']
    pred_mean = rep.surrogate_pred.mean
    pred_sd = jnp.sqrt(rep.surrogate_pred.variance)
    design_x = rep.design.X
    grid = rep.grid

    fig, ax = grid.plot(z=[exact, pred_mean, pred_sd],
                        titles=['exact', 'mean', 'sd'],
                        points=design_x,  max_cols=3)
    
    return (fig, ax)

In [ ]:
from uncprop.core.surrogate import Surrogate, SurrogateDistribution
from uncprop.core.samplers import mcmc_loop
from uncprop.core.distribution import Distribution, GaussianFromNumpyro
from numpyro.distributions import MultivariateNormal
from uncprop.models.vsem.experiment import VSEMReplicate
from uncprop.core.samplers import get_trace_plots

from uncprop.utils.distribution import _sample_gaussian_tril
from uncprop.core.samplers import (
    sample_distribution,
    init_rkpcn_kernel,
    _f_update_eup_cpm,
    _f_update_pcn_proposal,
    _pcn_proposal, 
    _mh_accept_reject, 
    _logZ_approx,
)

surrogate_tag = 'clip_gp'
key = jr.key(7543)
key, key_rep = jr.split(key)

rep = VSEMReplicate(key=key,
                    n_design=8,
                    noise_sd=1.0,
                    jitter=1e-6)

rep(surrogate_tag=surrogate_tag, n_mcmc=20_000)
_summarize_rep_gp(rep)
rep.density_comparison.plot(normalized=True, log_scale=True, max_cols=4)
rep.density_comparison.plot(normalized=True, log_scale=False, max_cols=4)

In [ ]:
samp = rep.mcmc_results['exact']

rep.grid.plot_kde(samp)

In [ ]:
rep.grid.plot_kde(rep.mcmc_results['rkpcn0'])

In [ ]:
from seaborn import kdeplot

key, key_grid_samp = jr.split(key)

samp = rep.density_comparison.sample_from_grid(key_grid_samp, 'exact', 1_000)


In [ ]:
rep.density_comparison.calc_wasserstein2('ep', 'eup',
                                          max_iterations=5_000, 
                                          epsilon=None)

In [ ]:
rep.density_comparison.calc_wasserstein2('mean', 'exact', max_iterations=5_000)

In [ ]:
rep.density_comparison.calc_wasserstein2('eup', 'mean')

In [ ]:
rep.density_comparison.grid.flat_grid.shape

In [ ]:
idx = jnp.argmax(rep.density_comparison.log_dens_grid['ep'])

rep.density_comparison.log_dens_grid['ep'][idx]

In [ ]:
rep.density_comparison.grid.flat_grid[idx]

In [ ]:
from uncprop.core.samplers import (
    init_adaptive_rwmh_kernel, AdaptationSettings, _proposal_tril_from_adaptation, AdaptationState
)

In [ ]:
key, key_init, key_samp = jr.split(key, 3)
initial_position = rep.posterior.prior.sample(key=key_init).squeeze()

samp, states, warmup_samp, prop_cov = sample_distribution(key_samp,
                                                          dist=rep.posterior,
                                                          initial_position=initial_position,
                                                          n_samples=50_000,
                                                          n_warmup=50_000,
                                                          thin_window=3)

In [ ]:
plot_2d_kde(samp, low=rep.posterior.support[0], high=rep.posterior.support[1])

In [ ]:
from uncprop.core.samplers import _init_dist_proposal_cov

prop_cov = _init_dist_proposal_cov(rep.posterior)

prop_cov

In [ ]:
key, key_position, key_ker, key_warmup = jr.split(key, 4)

logdensity = rep.posterior._get_log_density_function()
initial_position = rep.posterior.prior.sample(key=key_position).squeeze()
test = {}
adapt_settings = AdaptationSettings(**test)

initial_state, kernel = init_adaptive_rwmh_kernel(key=key_ker,
                                                  logdensity_fn=logdensity,
                                                  initial_position=initial_position,
                                                  adapt_settings=adapt_settings)

In [ ]:
samp = mcmc_loop(key_warmup, kernel, initial_state, 50_000)

In [ ]:
final_log_scale = samp.adapt_state.log_scale[-1]
final_cov = samp.adapt_state.cov_prop[-1]
final_adapt_state = AdaptationState(cov_prop=final_cov, log_scale=final_log_scale, times_adapted=0)
L = _proposal_tril_from_adaptation(final_adapt_state)

In [ ]:
plt.plot(samp.position[:,1])

In [ ]:
key, key_mcmc = jr.split(key, 2)
prop_cov = jnp.diag(jnp.array([1.0 ** 2, 2.0 ** 2]))
samp = rep.posterior.sample(key_mcmc, n=200_000, prop_cov=prop_cov)

get_trace_plots(trace={'av': samp[:,0], 'veg_init': samp[:,1]}, num_rows=1)

In [ ]:
plot_2d_kde(samp[100_000:], low=rep.posterior.support[0], high=rep.posterior.support[1])

In [ ]:
# log density
low, high = rep.posterior.support
upper_bound = lambda u: rep.posterior.prior.log_density(u) + rep.posterior.likelihood.log_density_upper_bound(u)

if surrogate_tag == 'clip_gp':
    def log_density(f, u):
        u = jnp.atleast_2d(u)
        upper = upper_bound(u)
        lp = jnp.clip(f, max=upper)
        lp = jnp.where(jnp.all((u >= low) & (u <= high), axis=1), lp, -jnp.inf)
        return lp.squeeze()

    # surrogate
    gp = rep.surrogate_posterior_clip_gp.surrogate
else:
    def log_density(f, u):
        u = jnp.atleast_2d(u)
        lp = f
        lp = jnp.where(jnp.all((u >= low) & (u <= high), axis=1), lp, -jnp.inf)
        return lp.squeeze()

    # surrogate
    gp = rep.surrogate_posterior_gp.surrogate


In [ ]:
# MCMC settings

from typing import NamedTuple

rho = 0.0
prop_cov = jnp.diag(jnp.array([0.2 ** 2, 1.0 ** 2]))

class UpdateInfo(NamedTuple):
    rho: float

f_update_info = UpdateInfo(rho=rho)

In [ ]:
key, key_init_pos, key_init_ker = jr.split(key, 3)
initial_position = rep.posterior.prior.sample(key_init_pos).squeeze()

initial_state, kernel = init_rkpcn_kernel(key=key_init_ker,
                                          log_density=log_density,
                                          gp=gp,
                                          initial_position=initial_position,
                                          u_prop_cov=prop_cov,
                                          f_update_fn=_f_update_pcn_proposal,
                                          f_update_info=f_update_info)

In [ ]:
key, key_sample = jr.split(key, 2)

out = mcmc_loop(key=key_sample,
                kernel=kernel,
                initial_state=initial_state,
                num_samples=200_000)

samp = out.position[100_000:]
samp_dict = {'av': samp[:,0], 'veg_init': samp[:,1]}

In [ ]:
plt.plot(out.logdensity)
get_trace_plots(trace=samp_dict, num_rows=1)

In [ ]:
fig, ax = plot_2d_kde(samp, low=low, high=high, contours=True, gridsize=200)
rep.density_comparison.plot(normalized=True, log_scale=False, max_cols=4)

### Old

In [ ]:
low, high = rep.posterior.support

def lp_sampler(key, u_curr, u_prop):
    u = jnp.stack([u_curr, u_prop], axis=0)
    lp = rep.surrogate_posterior_clip_gp.log_density(u).sample(key).squeeze()
    return jnp.where(jnp.all((u >= low) & (u <= high), axis=1), lp, -jnp.inf)

def lp_sample_ind(key, u_curr, u_prop):
    lp_curr = rep.surrogate_posterior_clip_gp.log_density(u_curr).sample(key).squeeze()
    lp_prop = rep.surrogate_posterior_clip_gp.log_density(u_prop).sample(key).squeeze()

    lp = jnp.array([lp_curr, lp_prop])
    u = jnp.stack([u_curr, u_prop], axis=0)
    return jnp.where(jnp.all((u >= low) & (u <= high), axis=1), lp, -jnp.inf)
    

In [ ]:
key, key_init_pos, key_init_ker, key_sample = jr.split(key, 4)

initial_position = rep.posterior.prior.sample(key_init_pos).squeeze()
prop_cov = jnp.diag(jnp.array([0.6 ** 2, 4.0 ** 2]))

initial_state, kernel = init_cut_kernel(key=key_init_ker,
                                        logp_sampler=lp_sampler,
                                        initial_position=initial_position,
                                        u_prop_cov=prop_cov)

In [ ]:
out = mcmc_loop(key=key_sample,
                kernel=kernel,
                initial_state=initial_state,
                num_samples=100_000)

samp = out.position[50_000:]
samp_dict = {'av': samp[:,0], 'veg_init': samp[:,1]}

In [ ]:
plt.plot(out.logdensity)
get_trace_plots(trace=samp_dict, num_rows=1)

In [ ]:
plot_2d_kde(samp, low=rep.posterior.support[0], high=rep.posterior.support[1], contours=True, gridsize=200)
rep.density_comparison.plot(normalized=True, log_scale=False, max_cols=4)

### Test rkpcn

In [ ]:
from uncprop.utils.distribution import _sample_gaussian_tril
from uncprop.core.samplers import _pcn_proposal, _mh_accept_reject, init_rkpcn_kernel

key, key_init_pos, key_sample = jr.split(key, 3)
initial_position = rep.posterior.prior.sample(key_init_pos).squeeze()
prop_cov = jnp.diag(jnp.array([0.2 ** 2, 1.0 ** 2]))

In [ ]:
out = mcmc_loop(key=key_sample,
                kernel=kernel,
                initial_state=initial_state,
                num_samples=500_000)

samp = out.position[100_000:]
samp_dict = {'av': samp[:,0], 'veg_init': samp[:,1]}

In [ ]:
plt.plot(out.logdensity)
get_trace_plots(trace=samp_dict, num_rows=1)

In [ ]:
test = (1, 2, (3,4))

a, b, (c, d) = test

print(a, b, c, d)

In [ ]:
plot_2d_kde(samp, low=low, high=high, contours=True, gridsize=200)
rep.density_comparison.plot(normalized=True, log_scale=False, max_cols=4)